### Yield From - Two-Way Communications

Recall...

In [1]:
def subgen():
    for i in range(10):
        yield i

We could consume the data from subgen in another generator in two different ways:

In [3]:
def delegator():
    for value in subgen():
        yield value

In [6]:
list(delegator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

or

In [7]:
def delegator():
    yield from subgen()

In [8]:
list(delegator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

The yield from has a lot more going on though!

With either definition we can call it this way:

In [ ]:
d = delegator()
next(d)
# etc...

What is going on exactly?

caller -> calls next(d)... this is passed along to the delegator

delegator -> calls yield from subgen... this is passed along to the subgen

subgen -> calls yield value...this yields a value back to the delegator which in turn yields the value back to the caller

So we have a 2-way communication

So if we can do next() and get yield back, what about send(), close(), and throw()?

The answer is yes!

#### How does the delegator behave when subgenerator returns?

it continues running normally

In [9]:
def delegator():
    yield from subgen()
    yield 'subgen closed'
    
def subgen():
    yield 1
    yield 2

In [11]:
d = delegator()

In [12]:
next(d)

1

In [13]:
next(d)

2

In [14]:
next(d)

'subgen closed'

In [15]:
next(d)

StopIteration: 

#### Inspecting the subgenerator

In [16]:
from inspect import getgeneratorlocals, getgeneratorstate

In [17]:
def delegator():
    a = 100
    s = subgen()
    yield from s
    yield 'subgen closed'

In [18]:
def subgen():
    yield 1
    yield 2

In [19]:
d = delegator()

In [20]:
getgeneratorstate(d)

'GEN_CREATED'

In [21]:
getgeneratorlocals(d)

{}

In [22]:
next(d)

1

In [23]:
getgeneratorstate(d)

'GEN_SUSPENDED'

In [24]:
getgeneratorlocals(d)

{'a': 100, 's': <generator object subgen at 0x00000295E2C850C8>}

In [25]:
s = getgeneratorlocals(d)['s']

In [26]:
getgeneratorstate(s)

'GEN_SUSPENDED'

In [27]:
getgeneratorlocals(s)

{}

In [28]:
next(d)

2

In [29]:
next(d)

'subgen closed'

In [30]:
getgeneratorstate(s)

'GEN_CLOSED'

In [31]:
getgeneratorstate(d)

'GEN_SUSPENDED'

In [32]:
next(d)

StopIteration: 

#### Code Examples

In [33]:
def squares(n):
    for i in range(n):
        yield i ** 2

In [34]:
def delegator(n):
    for value in squares(n):
        yield value

In [35]:
gen = delegator(5)

In [36]:
for _ in range(5):
    print(next(gen))

0
1
4
9
16


In [37]:
def delegator(n):
    yield from squares(n)

In [38]:
gen = delegator(5)
for _ in range(5):
    print(next(gen))

0
1
4
9
16


In [39]:
from inspect import getgeneratorstate, getgeneratorlocals

In [40]:
def song():
    yield "I'm a lumberjack and I'm OK"
    yield "I sleep all night and I work all day"

In [43]:
def play_song():
    count = 0
    s = song()
    yield from s
    yield 'song finished'
    print('player is exiting...')

In [44]:
player = play_song()

In [45]:
print(getgeneratorstate(player))
print(getgeneratorlocals(player))

GEN_CREATED
{}


In [46]:
next(player)

"I'm a lumberjack and I'm OK"

In [47]:
print(getgeneratorstate(player))
print(getgeneratorlocals(player))

GEN_SUSPENDED
{'count': 0, 's': <generator object song at 0x00000295E2D0FD48>}


In [48]:
s = getgeneratorlocals(player)['s']

In [49]:
print(getgeneratorstate(s))

GEN_SUSPENDED


In [50]:
next(player)

'I sleep all night and I work all day'

In [51]:
print(getgeneratorstate(player))
print(getgeneratorstate(s))

GEN_SUSPENDED
GEN_SUSPENDED


In [52]:
next(player)

'song finished'

In [53]:
print(getgeneratorstate(player))
print(getgeneratorstate(s))

GEN_SUSPENDED
GEN_CLOSED


In [54]:
next(player)

player is exiting...


StopIteration: 

In [55]:
print(getgeneratorstate(player))
print(getgeneratorstate(s))

GEN_CLOSED
GEN_CLOSED


In [57]:
def player():
    count = 1
    while True:
        print ('Run count:', count)
        yield from song()
        count += 1

In [58]:
p = player()

In [59]:
next(p), next(p)

Run count: 1


("I'm a lumberjack and I'm OK", 'I sleep all night and I work all day')

In [60]:
next(p), next(p)

Run count: 2


("I'm a lumberjack and I'm OK", 'I sleep all night and I work all day')

In [61]:
next(p), next(p)

Run count: 3


("I'm a lumberjack and I'm OK", 'I sleep all night and I work all day')